In [8]:
using LatticePhysics
using LatPhysPlottingPyPlot; using LatPhysReciprocalPlottingPyPlot;
using PyPlot; pygui(false);
using BenchmarkTools
using Traceur
using Random
using LinearAlgebra

In [9]:
function vector(
            bond    :: B,
            lattice :: LA
        ) :: Vector{Float64} where {N,L,B<:AbstractBond{L,N}, SU,BU,U<:AbstractUnitcell{SU,BU}, SL,LL,BL<:AbstractBond{LL,N},LA<:AbstractLattice{SL,BL,U}}

    # build the offset vector
    v = point(site(lattice,to(bond))) .- point(site(lattice,from(bond)))
    for i in 1:N
        v .+= wrap(bond)[i].*latticeVectors(lattice)[i]
    end
    # return the offset vector
    return v
end

function vector(
            bond     :: B,
            unitcell :: U
        ) :: Vector{Float64} where {N,L,B<:AbstractBond{L,N}, S,LU,BU<:AbstractBond{LU,N},U<:AbstractUnitcell{S,BU}}

    # build the offset vector
    v = point(site(unitcell,to(bond))) .- point(site(unitcell,from(bond)))
    for i in 1:N
        v .+= wrap(bond)[i].*latticeVectors(unitcell)[i]
    end
    # return the offset vector
    return v
end

vector (generic function with 2 methods)

In [14]:
import Random.shuffle!
function shuffle!(
            lattice :: L
        ) where {LB,N,LS,D,B<:AbstractBond{LB,N},S<:AbstractSite{LS,D},U,L<:AbstractLattice{S,B,U}}
    # make a new list of site indices
    indices_new = shuffle(collect(1:length(sites(lattice))))
    # make a new list of sites
    sites_new   = Vector{S}(undef, length(sites(lattice)))
    for s in 1:length(sites(lattice))
        sites_new[indices_new[s]] = site(lattice, s)
    end
    # reformat all bonds
    for b in bonds(lattice)
        from!(b, indices_new[from(b)])
        to!(b, indices_new[to(b)])
    end
    # set the sites
    sites!(lattice, sites_new)
    # shuffle all bonds
    bonds!(lattice, shuffle(bonds(lattice)))
end

shuffle! (generic function with 4 methods)

In [262]:
uc = getUnitcellSquareOctagon(4)
#uc = getUnitcellTriangular()
uc = getUnitcell_10_3_b(4)
#lt = getLatticeOpen(uc, 4)
lt = getLatticeByBondDistance(uc, 10)
relabelSitesBipartite!(lt)
shuffle!(lt)
lt_plot = deepcopy(lt)
for s in 1:length(sites(lt_plot))
    label!(site(lt_plot,s), s)
end
#relabelSitesBipartite!(lt)
#plotLattice(lt_plot, colorcode_bonds=:Kitaev, site_labels=true);
lt

Lattice object
--> type Lattice{Site{Int64,3},Bond{Int64,0},Unitcell{Site{Int64,3},Bond{Int64,3}}}
--> 608 sites of type Site{Int64,3}
--> 1554 bonds of type Bond{Int64,0}
--> unitcell of type Unitcell{Site{Int64,3},Bond{Int64,3}}

In [342]:
function constructUnitcellFromLatticeByBonds(
            lattice :: L
            ;
            check_site_labels  :: Bool = true,
            check_bond_labels  :: Bool = true,
            check_bond_vectors :: Bool = true,
            check_bonds_from   :: Bool = true,
            check_bonds_to     :: Bool = false,
            basis_site_min_copies :: Integer = 2,
            basis_site_min_copies_rel :: Real = 0.01,
            translations_from_bonds :: Bool = true,
            translations_from_sites :: Bool = true,
            precision :: Real = 1e-6
        ) where {LB,N,LS,D,B<:AbstractBond{LB,N},S<:AbstractSite{LS,D},U,L<:AbstractLattice{S,B,U}}
    
    
    ######
    # SYMMETRY EQUIVALENT OF ALL LATTICE SITES
    ######
    
    # identification table tab, tab[i] = j means i=j, initialized 0
    identification = zeros(Int64, length(sites(lattice)))
    # organized bonds list
    org_bonds_from = organizedBondsFrom(lattice)
    org_bonds_to   = organizedBondsTo(lattice)
    
    # go through all sites and find their symmetry equivalent
    for s1 in 1:length(sites(lattice))
        # the future equivalent site
        equiv_site = -1
        # check all sites that are already checked if they are equivalent
        for s2 in 1:s1-1
            # SITE LABEL
            if check_site_labels && label(site(lattice,s1)) != label(site(lattice,s2))
                # site labels do not agree
                continue
            end
            # BONDS FROM THAT SITE (OUTGOING)
            if check_bonds_from
                # check if outgoing bonds around s1 and s2 are the same
                org_bonds_from_1 = org_bonds_from[s1]
                org_bonds_from_2 = org_bonds_from[s2]
                if length(org_bonds_from_1) != length(org_bonds_from_2)
                    # numbers of bonds do not agree --> sites cannot agree
                    continue
                end
                # check all outgoing bonds if they agree
                # assume they do and find mismatches
                bonds_identical = true
                # iterate over all outgoing bonds b_from_1
                for b_from_1 in org_bonds_from_1
                    # check if b_from_1 can be found in org_bonds_from_2
                    b_found = false
                    for b_from_2 in org_bonds_from_2
                        if check_bond_labels && label(b_from_1) != label(b_from_2)
                            # labels of bonds do not agree, skip to next bond
                            continue
                        end
                        if check_bond_vectors && maximum(abs.(vector(b_from_1, lattice).-vector(b_from_2, lattice)))>precision
                            # vectors of bonds do not agree, skip to next bond
                            continue
                        end
                        # agreement wrt conducted tests, b_from_1 is in org_bonds_from_2
                        b_found = true
                        break
                    end
                    # if no agreement, b_from_1 is NOT in org_bonds_from_2
                    if !b_found
                        # the local bond environments cannot be the same
                        bonds_identical = false
                        break
                    end
                end
                if !bonds_identical
                    continue
                end
            end
            # bonds agree wrt conducted tests, sites can be identified
            equiv_site = s2
            break
        end
        # check if the site was found
        if equiv_site > 0
            identification[s1] = identification[equiv_site]
        else
            identification[s1] = s1
        end
        #println("$(s1) --> $(identification[s1])")
    end
    
    
    ######
    # CONSTRUCTION OF BASIS FROM LATTICE SITES
    ######
    
    # find all occuring sites
    occuring = zeros(Int64, length(identification))
    for i in identification
        occuring[i] += 1
    end
    basis_sites = Int64[
        i for i in unique(identification) if (occuring[i]>=basis_site_min_copies) && (occuring[i]>length(identification)*basis_site_min_copies_rel)
    ]
    println("Basis is: ", basis_sites)
    # assert that there are basis sites
    @assert length(basis_sites)>0 "No basis sites found!"
                
    # find all optimized basis sites
    identification_basis = copy(basis_sites)
    position_center = [sum([point(s)[alpha] for s in sites(lattice)])/length(sites(lattice)) for alpha in 1:length(point(site(lattice,1)))]
    # optimize all basis sites to be closest to proceeding sites and center
    for b in 1:length(basis_sites)
        # optimal version of basis site
        d_closest = Inf
        i_closest = 0
        # look at basis site b and find the closest to it
        for s in 1:length(sites(lattice))
            # check for the closest to center
            if identification[s] != basis_sites[b]
                continue
            end
            d = norm(position_center .- point(site(lattice,s)))
            for b2 in 1:b-1
                d += norm(point(site(lattice,basis_sites[b2])) .- point(site(lattice,s)))
            end
            if d<d_closest
                d_closest = d
                i_closest = s
            end
        end
        # set the site to the optimal copy
        basis_sites[b] = i_closest
    end
    println("Optimized Basis is: ", basis_sites)
            
                            
    ######
    # CONSTRUCTION OF LATTICE VECTORS
    ######
                
    # make a new list of sites
    new_sites = S[
                    deepcopy(site(lattice,s)) for s in basis_sites
                ]
                            
    # STEP 0 - obtain the distance WITHIN the unitcell
    uc_vectors = Matrix{Vector{Float64}}(undef, length(basis_sites), length(basis_sites))
    for i in 1:length(basis_sites)
        for j in 1:length(basis_sites)
            uc_vectors[i,j] = point(new_sites[j]) .- point(new_sites[i])
        end
    end
                            
    # STEP 1 - ALL LATTICE TRANSLATIONS, dependent on method
    translations = Vector{Float64}[]

    # insert all translations from bonds
    if translations_from_bonds
        # insert bonds
        for b in bonds(lattice)
            # check if correctly identifieable
            i_from = findfirst(c->c==identification[from(b)], identification_basis)
            i_to   = findfirst(c->c==identification[to(b)], identification_basis)
            if i_from == nothing || i_to == nothing
                continue
            end
            # find out the translation
            t = vector(b, lattice) .- uc_vectors[i_from, i_to]
            # check if it is already in the list
            found = false
            for tp in translations
                if sum(abs.(t.-tp)) < precision
                    found=true
                    break
                end
            end
            # add it to the list of translations
            if found == false
                push!(translations, t)
            end
        end
    end

    # insert all translations from sites
    if translations_from_sites
        # insert bonds
        for s in 1:length(sites(lattice))
            # check if correctly identifieable
            i = findfirst(c->c==identification[s], identification_basis)
            if i == nothing
                continue
            end
            # find out the translation by subtracting the difference to basis site 1
            t = point(site(lattice,s)) .+ uc_vectors[i, 1] .- point(site(lattice,basis_sites[1]))
            # check if it is already in the list
            found = false
            for tp in translations
                if sum(abs.(t.-tp)) < precision
                    found=true
                    break
                end
            end
            # add it to the list of translations
            if found == false
                push!(translations, t)
            end
        end
    end

    # plot and scatter translations
    println("$(length(translations)) translations found")
    #scatter([t[1] for t in translations], [t[2] for t in translations])
    #scatter3D([t[1] for t in translations], [t[2] for t in translations], [t[3] for t in translations])
                            
                            
                            
                            
    # STEP 2 - CONSTRUCT a_j BASED ON TRANSLATIONS

    # make a list which contains both length and translation
    translations_list = Tuple{Float64, Vector{Float64}}[(norm(t), t) for t in translations if norm(t)>1e-4]
    # sort the list
    sort!(translations_list, by=e->e[1])
    #println.(translations_list)

    # make a list of aj
    aj = Vector{Float64}[]

    # iterate the following as long as there are unresolved translations
    while length(translations_list) > 0
        # pick the first translation and define it to be a new lattice vector
        new_aj = translations_list[1][2]
        # push the lattice vector
        push!(aj, new_aj)
        # shorten the list
        translations_list = translations_list[2:end]
        # build combinations along which the other translations can be shifted
        aj_combs = unique([(p1*aj[i1].+p2*aj[i2].+p3*aj[i3]) for i1 in 1:length(aj) for i2 in 1:length(aj) for i3 in 1:length(aj) for p1 in [-1,0,1] for p2 in [-1,0,1] for p3 in [-1,0,1]])
        #println(length(aj_combs))
        # move all translations closest to origin along new_aj
        for t in translations_list
            # asumme it was relocated
            relocated = true
            # as long as it cuold be relocated in a step, try moving it along the possible combinations
            while relocated
                # asumme not relocated
                relocated = false
                # try different combinations
                for c in aj_combs
                    if norm(t[2].+c) < norm(t[2])
                        t[2] .+= c
                        relocated = true
                        break
                    end
                end
            end
        end
        # define a new list of translations by recalculating the distance
        translations_list = Tuple{Float64, Vector{Float64}}[(norm(t[2]), t[2]) for t in translations_list if norm(t[2])>1e-8]
        sort!(translations_list, by=e->e[1])
    end

                                                                        
                                                                        
    # STEP 3 - EXPRESS BONDS IN TERMS OF BRAVAIS LATTICE VECTORS

    # find out the new BL dimension
    new_N = length(aj)

    # list with old bonds
    old_bonds = Bond{LB,N}[
                    b for b in bonds(lattice) if from(b) in basis_sites
                ]

    # find out which BLV combinations the bonds are having 
    BLV_comb = Vector{NTuple{new_N, Int64}}(undef, length(old_bonds))
    # combination of ajs
    aj_combs = Tuple{Int64,Int64,Int64,Int64,Int64,Int64,Vector{Float64}}[(p,i,q,j,r,k,p*aj[i].+q*aj[j].+r*aj[k]) for i in 1:length(aj) for j in 1:length(aj) for k in 1:length(aj) for r in [-1,0,1] for q in [-1,0,1] for p in [-1,0,1]]
    i_list = zeros(Int64, length(old_bonds))
    j_list = zeros(Int64, length(old_bonds))
    # find out by taking scalar product with perp vector
    for bi in 1:length(old_bonds)
        # find out the pure inter-unitcell vector for the bond
        b = old_bonds[bi]
        # check if correctly identifieable
        i_from = findfirst(c->c==identification[from(b)], identification_basis)
        i_to   = findfirst(c->c==identification[to(b)], identification_basis)
        if i_from == nothing || i_to == nothing
            error("the basis connects to a site which is not in replicas of the basis")
        end
        # save i and j
        i_list[bi] = i_from
        j_list[bi] = i_to
        # find out the translation vector that has to be expressed in aj
        v = vector(b, lattice) .- uc_vectors[i_from, i_to]
        # try to shift the vector accordingly and record the shifts
        relocated = true
        wrap = zeros(Int64, new_N)
        # as long as it cuold be relocated in a step, try moving it along the possible combinations
        while relocated
            # asumme not relocated
            relocated = false
            # try different combinations
            for c in aj_combs
                if norm(v.+c[end]) < norm(v)
                    v .+= c[end]
                    relocated = true
                    wrap[c[2]] -= c[1]
                    wrap[c[4]] -= c[3]
                    wrap[c[6]] -= c[5]
                    break
                end
            end
        end
        if norm(v) > precision
            error("could not fully fold back the following point: $(v)")
        end         
        # save the wrap
        BLV_comb[bi] = NTuple{new_N,Int64}(wrap)
    end

    # make new bonds
    new_bonds = unique(Bond{LB,new_N}[
        newBond(
            Bond{LB,new_N},
            i_list[b],
            j_list[b],
            label(old_bonds[b]),
            BLV_comb[b]
        ) for b in 1:length(old_bonds)
    ])

    # make a new unitcell
    uc = newUnitcell(
        Unitcell{S,Bond{LB,new_N}},
        aj,
        new_sites,
        new_bonds                                                                                                    
    )
                                                                                                        
    # return the unitcell
    return uc                        
end

constructUnitcellFromLatticeByBonds (generic function with 1 method)

In [343]:
ucc = constructUnitcellFromLatticeByBonds(lt, check_site_labels=true, basis_site_min_copies_rel=0.08, translations_from_sites=false, translations_from_bonds=true)
pygui(true)
plotLattice(getLatticeOpen(ucc, 1), site_labels=true, colorcode_bonds=:Kitaev);
plotLattice(getLatticeOpen(ucc, 2), site_labels=true, colorcode_bonds=:Kitaev);
bonds(ucc)

Basis is: [1, 2]
Optimized Basis is: [1, 2]
5 translations found


6-element Array{Bond{Int64,2},1}:
 Bond{Int64,2} 2-->1 @(0, 0): 1 
 Bond{Int64,2} 1-->2 @(0, 0): 1 
 Bond{Int64,2} 1-->2 @(-1, 0): 2
 Bond{Int64,2} 1-->2 @(0, -1): 3
 Bond{Int64,2} 2-->1 @(1, 0): 2 
 Bond{Int64,2} 2-->1 @(0, 1): 3 

In [228]:
bonds(getUnitcellSquareOctagon())

12-element Array{Bond{Int64,2},1}:
 Bond{Int64,2} 1-->2 @(0, 0): 1 
 Bond{Int64,2} 2-->4 @(0, 0): 1 
 Bond{Int64,2} 4-->3 @(0, 0): 1 
 Bond{Int64,2} 3-->1 @(0, 0): 1 
 Bond{Int64,2} 2-->1 @(0, 0): 1 
 Bond{Int64,2} 4-->2 @(0, 0): 1 
 Bond{Int64,2} 3-->4 @(0, 0): 1 
 Bond{Int64,2} 1-->3 @(0, 0): 1 
 Bond{Int64,2} 3-->2 @(0, -1): 1
 Bond{Int64,2} 2-->3 @(0, 1): 1 
 Bond{Int64,2} 4-->1 @(-1, 0): 1
 Bond{Int64,2} 1-->4 @(1, 0): 1 

In [250]:
uc

Unitcell object
--> type Unitcell{Site{Int64,3},Bond{Int64,3}}
--> 6 sites of type Site{Int64,3}
--> 18 bonds of type Bond{Int64,3}

In [267]:
function rotateAroundZAxis!(lattice::AbstractLattice, angle::Real)
    # rotate the lattice vectors
    for lv in latticeVectors(lattice)
        # rotate and overwrite simulateously all components
        lv[1], lv[2] = cos(angle)*lv[1] + sin(angle)*lv[2]  ,   -sin(angle)*lv[1] + cos(angle)*lv[2]
    end
    # rotate the basis sites
    for p in point.(sites(lattice))
        # rotate and overwrite simulateously all components
        p[1], p[2] = cos(angle)*p[1] + sin(angle)*p[2]  ,   -sin(angle)*p[1] + cos(angle)*p[2]
    end
end

function rotateAroundZAxisDeg!(lattice::AbstractLattice, angle::Real)
    rotateAroundZAxis!(lattice, angle*pi/180.0)
end
export rotateAroundZAxisDeg!



In [329]:
m = 3
n = m+1

uc = getUnitcellHoneycomb(4)
lt = getLatticeByBondDistance(uc, 12*m)
#plotLattice(lt)

ltp = deepcopy(lt)
rotateAroundZAxis!(ltp, acos(0.5*(m*m + n*n + 4*m*n)/(m*m + n*n + m*n)))

In [330]:
ltc = deepcopy(ltp)
bonds_new = deepcopy(bonds(lt))
for b in bonds_new
    from!(b, from(b)+length(sites(ltc)))
    to!(b, to(b)+length(sites(ltc)))
end
append!(sites(ltc), deepcopy(sites(lt)))
append!(bonds(ltc), bonds_new)
#plotLattice(ltc);
ltc

Lattice object
--> type Lattice{Site{Int64,2},Bond{Int64,0},Unitcell{Site{Int64,2},Bond{Int64,2}}}
--> 3998 sites of type Site{Int64,2}
--> 11664 bonds of type Bond{Int64,0}
--> unitcell of type Unitcell{Site{Int64,2},Bond{Int64,2}}

In [331]:
ltg = deepcopy(ltc)
LB = typeof(label(bonds(ltc)[1]))
bonds_geometry = Bond{LB, 0}[
    newBond(
        Bond{LB, 0},
        i,j,
        getDefaultLabelN(LB, round(Int64, 2*norm(point(site(ltg,i)).-point(site(ltg,j))))),
        NTuple{0,Int64}()
    ) for i in 1:length(sites(ltg)) for j in 1:length(sites(ltg)) if norm(point(site(ltg,i)).-point(site(ltg,j)))<1
]
bonds!(ltg, bonds_geometry)

53186-element Array{Bond{Int64,0},1}:
 Bond{Int64,0} 1-->1 @(): 0      
 Bond{Int64,0} 1-->2 @(): 1      
 Bond{Int64,0} 1-->3 @(): 1      
 Bond{Int64,0} 1-->4 @(): 1      
 Bond{Int64,0} 1-->2000 @(): 0   
 Bond{Int64,0} 1-->2001 @(): 1   
 Bond{Int64,0} 1-->2002 @(): 1   
 Bond{Int64,0} 1-->2003 @(): 1   
 Bond{Int64,0} 1-->2004 @(): 2   
 Bond{Int64,0} 1-->2005 @(): 2   
 Bond{Int64,0} 1-->2006 @(): 2   
 Bond{Int64,0} 1-->2008 @(): 2   
 Bond{Int64,0} 2-->1 @(): 1      
 ⋮                               
 Bond{Int64,0} 3996-->3889 @(): 1
 Bond{Int64,0} 3996-->3996 @(): 0
 Bond{Int64,0} 3997-->1924 @(): 1
 Bond{Int64,0} 3997-->3785 @(): 2
 Bond{Int64,0} 3997-->3889 @(): 1
 Bond{Int64,0} 3997-->3890 @(): 1
 Bond{Int64,0} 3997-->3997 @(): 0
 Bond{Int64,0} 3997-->3998 @(): 2
 Bond{Int64,0} 3998-->3890 @(): 1
 Bond{Int64,0} 3998-->3925 @(): 2
 Bond{Int64,0} 3998-->3997 @(): 2
 Bond{Int64,0} 3998-->3998 @(): 0

In [332]:
organizedBondsFrom(ltg)[rand(1:length(sites(ltg)))]

13-element Array{Bond{Int64,0},1}:
 Bond{Int64,0} 3443-->1543 @(): 1
 Bond{Int64,0} 3443-->1545 @(): 2
 Bond{Int64,0} 3443-->1640 @(): 2
 Bond{Int64,0} 3443-->1642 @(): 1
 Bond{Int64,0} 3443-->1741 @(): 2
 Bond{Int64,0} 3443-->1743 @(): 1
 Bond{Int64,0} 3443-->1846 @(): 2
 Bond{Int64,0} 3443-->3263 @(): 2
 Bond{Int64,0} 3443-->3350 @(): 1
 Bond{Int64,0} 3443-->3352 @(): 1
 Bond{Int64,0} 3443-->3443 @(): 0
 Bond{Int64,0} 3443-->3538 @(): 1
 Bond{Int64,0} 3443-->3637 @(): 2

In [341]:
ucc = constructUnitcellFromLatticeByBonds(
    ltg, 
    check_site_labels=false,
    basis_site_min_copies=4,
    translations_from_sites=true,
    translations_from_bonds=false,
    precision = 1e-2
)

Basis is: Int64[]


AssertionError: AssertionError: No basis sites found!